In [1]:
import board_2d as b2d
import carcassonne as carc

In [2]:
board = b2d.Board()
gs = carc.Game_state(board)
gs.set_initial_state()

played
adding (0, 1)  to  (c_val:2_owner:None)
(c_val:2_owner:None)
adding (-1, 0)  to  (p_val:1_owner:None)
(p_val:1_owner:None)
adding (1, 0)  to  (p_val:1_owner:None)
(p_val:1_owner:None)
adding (-1, 0)  to  (f_val:0_owner:None)
(f_val:0_owner:None)
adding (1, 0)  to  (f_val:0_owner:None)
(f_val:0_owner:None)
adding (0, -1)  to  (f_val:0_owner:None)
(f_val:0_owner:None)
adding (-1, 0)  to  (f_val:0_owner:None)
(f_val:0_owner:None)
adding (1, 0)  to  (f_val:0_owner:None)
(f_val:0_owner:None)


In [4]:
#print(gs.board.filled_locations[(0,0)].tile)
for coord,loc in gs.board.available_locations.items():
    print(loc)
    

(-1, 0): {'rd': '(f_val:0_owner:None)', 'r': '(p_val:1_owner:None)', 'ru': '(f_val:0_owner:None)'}
(0, 1): {'d': '(c_val:2_owner:None)'}
(1, 0): {'lu': '(f_val:0_owner:None)', 'l': '(p_val:1_owner:None)', 'ld': '(f_val:0_owner:None)'}
(0, -1): {'u': '(f_val:0_owner:None)'}


In [ ]:
print(gs.board)